<a href="https://colab.research.google.com/github/youavang/Covid-19_CT_Scan_With_Deep_Learning/blob/main/ResNet152_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Down grade to tensorflow to 2.2.0 version in order to use tf-explain Grad CAM
!pip install tensorflow==2.2.0

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!nvidia-smi #show the allocated GPU

In [ ]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Input, AveragePooling2D, Reshape, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,TimeDistributed, LSTM, concatenate
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator,save_img, load_img, img_to_array
from keras.initializers import RandomNormal
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
import keras.backend as k
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import datetime
from keras.callbacks import TensorBoard
from keras.applications import ResNet152, InceptionV3, Xception, VGG16, VGG19
from keras.applications.resnet_v2 import ResNet50V2
from keras.utils.vis_utils import plot_model

In [ ]:
#Connect to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Add the COVID-CTset to your drive through this link:
# https://drive.google.com/drive/folders/1xdk-mCkxCDNwsMAk2SGv203rY1mrbnPB?usp=sharing

In [ ]:
#Install essential libraries
!pip install zipfile36

In [ ]:
archive = zipfile.ZipFile("/content/drive/MyDrive/Train&Validation.zip") #Path to the shared data for training and validation
for file in archive.namelist():
     archive.extract(file, './data') #Extract the data

In [ ]:
fold_num=1 #Select Fold Number
shape=(512,512,1) #shape of the dataset images (in TIFF format)
num_class=2 #Number of classes (normal and COVID-19)

In [ ]:
#Here we set the data generators for applying data augmentation methods
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,zoom_range=0.05,rotation_range=360,width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05)
test_datagen = ImageDataGenerator()
train_df =pd.read_csv('/content/drive/MyDrive/CSV/train{}.csv'.format(fold_num)) #read train csv file
validation_df = pd.read_csv('/content/drive/MyDrive/CSV/validation{}.csv'.format(fold_num)) #read validation csv file (Validation in the training process)
train_df = shuffle(train_df) #Shuffle the train data
test_df = pd.read_csv('/content/drive/MyDrive/CSV/test{}.csv'.format(fold_num))#read test csv file (For evaluating the final version of the trained network)

In [ ]:
#Create the generators
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='data',
      x_col="filename",
      y_col="class",
      target_size=shape[:2],
      batch_size=10,
      class_mode='categorical',color_mode="grayscale",shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='data',
        x_col="filename",
        y_col="class",
        target_size=shape[:2],
        batch_size=10,
        class_mode='categorical',color_mode="grayscale",shuffle=True)
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='data',
        x_col="filename",
        y_col="class",
        target_size=shape[:2],
        batch_size=10,
        class_mode='categorical',color_mode="grayscale",shuffle=False)

In [ ]:
k.clear_session() #Clear keras backend 
try:
  os.mkdir('models') #create folder for saving the trained networks
except:
  pass
full_name='ResNet152-FPN-fold{}'.format(fold_num)

input_tensor=Input(shape=shape)
weight_model = ResNet152(weights='imagenet', include_top=False) #Load ResNet152 ImageNet pre-trained weights
weight_model.save_weights('weights.h5') #Save the weights
base_model = ResNet152(weights=None, include_top=False, input_tensor=input_tensor) #Load the ResNet152 model without weights
base_model.load_weights('weights.h5',skip_mismatch=True, by_name=True) #Load the ImageNet weights on the ResNet model except the first layer(because the first layer has one channel in our case)

In [ ]:
# Attaching our own FC

x = base_model.output
x = AveragePooling2D(pool_size = (2,2), padding = 'same', strides=(1,1))(x)
x = Flatten(name = 'flatten_layer')(x)
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)
output=Dense(num_class, activation='softmax',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(x)
model=Model(base_model.input, output)

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='resnet152_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
for layer in model.layers:
  layer.trainable = True
model.compile(optimizer=optimizers.Nadam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name  # Path to save the trained models
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history=model.fit(train_generator,steps_per_epoch=100, epochs=100, validation_data=validation_generator, shuffle=True,callbacks=[callbacks_list, tensorboard_callback]) #start training

In [ ]:
# save entire model and weights
model.save('Resnet152_model.h5')

In [ ]:
# load model and weights
#model=load_model('/content/drive/MyDrive/trained_models/resnet152_lstm_model.hdf5')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
# summarize history for accuracy

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
score = model.evaluate(train_generator)
print("Accuracy:%.2f%%" % (score[1]*100))

In [ ]:
score = model.evaluate(test_generator)
print("Accuracy:%.2f%%" % (score[1]*100))

In [ ]:
y_pred=model.predict(test_generator)
ypred=np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_generator.classes, ypred)
cm

In [ ]:
import seaborn as sns
sns.heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Blues')

In [ ]:
class_labels=list(test_generator.class_indices.keys())

In [ ]:
from sklearn.metrics import classification_report
# class_name=['Covid','Normal']
print(classification_report(test_generator.classes, ypred, target_names=class_labels))

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# make a prediction
Y_pred = model.predict(test_generator, test_generator.samples // test_generator.batch_size+1) #(test_gen, steps=len(df_val), verbose=1)
Ypred = np.argmax(y_pred, axis=1)
fpr, tpr, thresholds = roc_curve(test_generator.classes, Ypred)
auc_area = auc(fpr, tpr)


plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='area = {:.3f}'.format(auc_area))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
!pip install scikit-plot

In [ ]:
true_classes=test_generator.classes

In [ ]:
%matplotlib inline  
import scikitplot as skplt

[print(k, ":", v) for k,v in enumerate(class_labels)]
true_map_classes = [class_labels[x] for x in true_classes]
predicted_map_classes = [class_labels[x] for x in ypred]

skplt.metrics.plot_confusion_matrix(
    true_map_classes, 
    predicted_map_classes,
    labels=class_labels,
    x_tick_rotation=90,
    figsize=(6,6))

In [ ]:
skplt.metrics.plot_precision_recall(true_classes, y_pred, figsize=(6,6))

In [ ]:
skplt.metrics.plot_roc(true_classes, y_pred, figsize=(6,6))

In [ ]:
#Model Evaluation
trained_models=[]
for r,d,f in os.walk('models'): #Take the path to the trained nets 
  for file in f:
    if '.hdf5' in file:
      trained_models.append(os.path.join(r,file))

reports=[]
for trn_model in trained_models: #evaluate the network on each trained net
  k.clear_session()
  netpath=trn_model 
  models_name=trn_model
  fold_num=trn_model[trn_model.index('fold')+4] #find the fold number
  net=keras.models.load_model(netpath) #load model

  covid_label= test_generator.class_indices['covid'] #get the index of COVID-19 class 
  normal_label= test_generator.class_indices['normal']  #get the index of normal class 
  tp=0 #True Positives
  fp=0 #False Positives
  anum=0 #All the images numbers
  ###########
  ctp=0 #Correct classified COVID-19 cases
  cfp=0 #Wrong classified COVID-19 cases
  cfn=0 #Not classified COVID-19 cases
  ctn=0 #Correctly not classified COVID-19 cases
  cnum=0 #Number of COVID cases
  ################
  ntp=0 #Correct classified normal cases
  nfp=0 #Wrong classified normal cases
  nfn=0 #Not classified normal cases
  ntn=0 #Correctly not classified normal cases
  nnum=0 #Number of normal cases
  for num,img_name in enumerate(test_generator.filenames): #load image
    gt_ind=test_generator.classes[num] #get the loaded image class index
    img=cv2.imread(os.path.join('data',img_name),cv2.IMREAD_UNCHANGED) #load image
    pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #get the predicted class index
    anum+=1 #count the number of images
    if gt_ind==covid_label:
      cnum+=1
      if pred_ind==covid_label:
        tp+=1
        ctp+=1
        ntn+=1
      else:
        fp+=1
        nfp+=1
        cfn+=1
    elif gt_ind==normal_label:
      nnum+=1
      if pred_ind==normal_label:
        ctn+=1
        ntp+=1
        tp+=1
      else:
        cfp+=1
        nfn+=1
        fp+=1


  overall_acc=tp/(tp+fp) #overall accuracy
  cacc=(ctp+ctn)/(ctp+ctn+cfp+cfn) #covid accurayc
  nacc=(ntp+ntn)/(ntp+ntn+nfp+nfn) #normal accuracy
  csens=ctp/(ctp+cfn) #covid sensitivity
  nsens=ntp/(ntp+nfn) #normal sensitivity
  cspec=ctn/(ctn+cfp) #covid specificity
  nspec=ntn/(ntn+nfp) #normal specificity
  cprec=ctp/(ctp+cfp) #covid precision
  nprec=ntp/(ntp+nfp) #normal precision

  reports.append([models_name,fold_num,tp,fp,ctp,cfn,cfp,ntp,nfn,nfp,overall_acc,cacc,nacc,csens,nsens,cspec,nspec,cprec,nprec])


  print(models_name)
  print('tp: ',tp,'fp: ',fp)

with open('FPN.csv', mode='w',newline='') as csv_file:
    csvwriter = csv.writer(csv_file, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['models_name','fold_num','tp','fp','ctp','cfn','cfp','ntp','nfn','nfp','overall_acc','cacc','nacc','csens','nsens','cspec','nspec','cprec','nprec'])
    for row in reports:
        csvwriter.writerow(row)  